So NER needs to have token and sub token level tokenization and understanding so we use custom function and also used custom metrics eval and sequel as we need to check sub token level of accuracy done in QA models as well


In [ ]:
!pip install datasets==3.6.0 --force-reinstall
#

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install evaluate seqeval

  Using cached seqeval-1.2.2-py3-none-any.whl


In [ ]:
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
import evaluate

In [ ]:
dataset = load_dataset("conll2003") # loading the dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

conll2003.py: 0.00B [00:00, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
label_list = dataset["train"].features["ner_tags"].feature.names # extracting features from this dataset for feeding to the model
id2label = {i: label for i, label in enumerate(label_list)} #giving it id2 and labeli2
label2id = {label: i for i, label in enumerate(label_list)}


In [ ]:
#setting checkpoint as untrained bert
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#tokenizing each word and subwords
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,  # Important for word-level tasks
        padding="max_length",
        max_length=128
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignore these positions
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                # For sub-tokens, you can either repeat the label
                # or set -100 to ignore them in loss
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
#batching and tokenizing all data in dataset
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [ ]:
#defining the model and setting all checkpoint model and label/logits
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)#establising the collator wont be used as we padded it to max len in tokenization but still

In [ ]:
metric = evaluate.load("seqeval") #as we need custom metric for subword anaylsis we cant use only numpy we use this as well

In [ ]:
#we use metric function called seqeual which we imported above to use  in fucntion compute metric for checking subword accurracy in all cases
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (-100) for true labels and predictions
    true_labels = [
        [id2label[l] for l in label if l != -100]
        for label in labels
    ]
    true_preds = [
        [id2label[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# base Training setup
training_args = TrainingArguments(
    output_dir="./ner-checkpoint",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    #fp16=torch.cuda.is_available(),
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    push_to_hub=False,
    report_to="none",
    logging_steps=50
)

In [ ]:
# basic Trainer functions and agrs
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-3178454214.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#final training
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.055700,0.051741,0.925715,0.931638,0.928667,0.986132
2,0.026600,0.050823,0.932116,0.945614,0.938816,0.987243
3,0.020900,0.051216,0.936979,0.946287,0.941610,0.987983


TrainOutput(global_step=2634, training_loss=0.06125767724751159, metrics={'train_runtime': 982.1828, 'train_samples_per_second': 42.887, 'train_steps_per_second': 2.682, 'total_flos': 2751824963545344.0, 'train_loss': 0.06125767724751159, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate(tokenized_dataset["test"])
print(results)

{'eval_loss': 0.11444572359323502, 'eval_precision': 0.8863518422418267, 'eval_recall': 0.9075451647183846, 'eval_f1': 0.8968233132055659, 'eval_accuracy': 0.9789768443726441, 'eval_runtime': 24.499, 'eval_samples_per_second': 140.944, 'eval_steps_per_second': 8.817, 'epoch': 3.0}


In [ ]:
#saving model and adding config for id2label in model
trainer.save_model("./NER-distilbert-finetuned-custom")
tokenizer.save_pretrained("./NER-distilbert-finetuned-custom")
model.config.id2label = id2label
model.config.label2id = label2id

('./NER-distilbert-finetuned-custom/tokenizer_config.json',
 './NER-distilbert-finetuned-custom/special_tokens_map.json',
 './NER-distilbert-finetuned-custom/vocab.txt',
 './NER-distilbert-finetuned-custom/added_tokens.json',
 './NER-distilbert-finetuned-custom/tokenizer.json')

In [ ]:
hf_code = ''

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("./NER-distilbert-finetuned-custom")
tokenizer = AutoTokenizer.from_pretrained("./NER-distilbert-finetuned-custom")

# Push to Hub (your username/repo_name)
repo_name = "Noobhacker69/NER-distilbert-finetuned"  # you choose the name
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Noobhacker69/NER-distilbert-finetuned/commit/6c853351b8f05f9d1bf3db5e54a3b445a5b886d9', commit_message='Upload tokenizer', commit_description='', oid='6c853351b8f05f9d1bf3db5e54a3b445a5b886d9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Noobhacker69/NER-distilbert-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Noobhacker69/NER-distilbert-finetuned'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
mnameinf = 'Noobhacker69/NER-distilbert-finetuned'
model = AutoModelForTokenClassification.from_pretrained(mnameinf)
tokenizer = AutoTokenizer.from_pretrained(mnameinf)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
input='Hugging Face was founded in Paris by Julien Chaumond.'

In [ ]:
tokenized = tokenizer(input,
          padding=True,
          truncation=True,
          return_tensors="pt",)#putting padding cuz inferece and truncation

In [ ]:
with torch.no_grad():
  tokenwithlogits = model(**tokenized)#no graident calculation and getting logits and results

In [ ]:
tokenwithlogits

TokenClassifierOutput(loss=None, logits=tensor([[[ 9.0921, -0.9132, -1.4372, -0.7800, -0.9474, -0.5659, -0.7696,
          -0.9978, -1.0889],
         [-0.3923,  1.1802, -2.0384,  4.8663, -1.7768,  0.1385, -2.1369,
           0.6245, -2.1418],
         [ 0.8769, -1.6000, -0.2680, -0.3183,  5.4767, -1.5505, -0.8979,
          -2.2523, -0.4133],
         [ 7.8669, -1.7274, -1.4732, -0.8619,  1.5679, -1.3786, -0.6423,
          -2.0718, -0.6811],
         [ 8.7131, -1.4320, -1.7222, -0.5167,  0.3809, -1.0770, -0.8794,
          -1.3706, -1.2866],
         [ 8.9517, -1.5011, -1.8404, -0.9545, -0.3054, -0.6821, -0.7103,
          -1.5513, -1.2707],
         [-0.5956, -0.7878, -1.4985,  0.3490, -1.1584,  7.4845, -0.2393,
          -0.8597, -1.7415],
         [ 8.8037, -0.6030, -1.5508, -1.1640, -0.3858, -1.1249, -1.0119,
          -1.3431, -1.2752],
         [-1.3599,  7.1974, -0.8212, -0.0376, -1.7612, -0.5614, -1.4725,
          -0.7433, -2.1703],
         [-0.8860, -0.1919,  7.6458, -1.58

In [ ]:
logits =  tokenwithlogits.logits #getting logits from data
logits

tensor([[[ 9.0921, -0.9132, -1.4372, -0.7800, -0.9474, -0.5659, -0.7696,
          -0.9978, -1.0889],
         [-0.3923,  1.1802, -2.0384,  4.8663, -1.7768,  0.1385, -2.1369,
           0.6245, -2.1418],
         [ 0.8769, -1.6000, -0.2680, -0.3183,  5.4767, -1.5505, -0.8979,
          -2.2523, -0.4133],
         [ 7.8669, -1.7274, -1.4732, -0.8619,  1.5679, -1.3786, -0.6423,
          -2.0718, -0.6811],
         [ 8.7131, -1.4320, -1.7222, -0.5167,  0.3809, -1.0770, -0.8794,
          -1.3706, -1.2866],
         [ 8.9517, -1.5011, -1.8404, -0.9545, -0.3054, -0.6821, -0.7103,
          -1.5513, -1.2707],
         [-0.5956, -0.7878, -1.4985,  0.3490, -1.1584,  7.4845, -0.2393,
          -0.8597, -1.7415],
         [ 8.8037, -0.6030, -1.5508, -1.1640, -0.3858, -1.1249, -1.0119,
          -1.3431, -1.2752],
         [-1.3599,  7.1974, -0.8212, -0.0376, -1.7612, -0.5614, -1.4725,
          -0.7433, -2.1703],
         [-0.8860, -0.1919,  7.6458, -1.5865,  0.1754, -1.0684, -0.3731,
         

In [ ]:
data = torch.argmax(logits ,dim=-1)
data# max arg logits and getting results

tensor([[0, 3, 4, 0, 0, 0, 5, 0, 1, 2, 2, 2, 0, 8]])

In [ ]:
id2label = model.config.id2label
label2id = model.config.label2id # calling id2 labels from config model

print(id2label)
print(label2id)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}
{'B-LOC': 5, 'B-MISC': 7, 'B-ORG': 3, 'B-PER': 1, 'I-LOC': 6, 'I-MISC': 8, 'I-ORG': 4, 'I-PER': 2, 'O': 0}


In [ ]:
pred_labels = [id2label[i.item()] for i in data[0]]# adding labels array/tensor

In [ ]:
print(list(zip(tokenizer.convert_ids_to_tokens(tokenized["input_ids"][0]), pred_labels))) #using zip to print predlabels and outputs

[('[CLS]', 'O'), ('hugging', 'B-ORG'), ('face', 'I-ORG'), ('was', 'O'), ('founded', 'O'), ('in', 'O'), ('paris', 'B-LOC'), ('by', 'O'), ('julien', 'B-PER'), ('cha', 'I-PER'), ('##um', 'I-PER'), ('##ond', 'I-PER'), ('.', 'O'), ('[SEP]', 'I-MISC')]


In [ ]:
from transformers import pipeline

ner = pipeline('ner',model='Noobhacker69/NER-distilbert-finetuned',tokenizer='Noobhacker69/NER-distilbert-finetuned',aggregation_strategy='simple')# using agr startery for combined

Device set to use cpu


In [ ]:
output = ner('Hugging Face was founded in Paris by Julien Chaumond.')#agregated results
output

[{'entity_group': 'ORG',
  'score': np.float32(0.96154726),
  'word': 'hugging face',
  'start': 0,
  'end': 12},
 {'entity_group': 'LOC',
  'score': np.float32(0.9975641),
  'word': 'paris',
  'start': 28,
  'end': 33},
 {'entity_group': 'PER',
  'score': np.float32(0.9971682),
  'word': 'julien chaumond',
  'start': 37,
  'end': 52}]

In [ ]:
output = ner('Hugging Face was founded in Paris by Julien Chaumond.')#non agregated results
output

[{'entity': 'B-ORG',
  'score': np.float32(0.94551903),
  'index': 1,
  'word': 'hugging',
  'start': 0,
  'end': 7},
 {'entity': 'I-ORG',
  'score': np.float32(0.97757554),
  'index': 2,
  'word': 'face',
  'start': 8,
  'end': 12},
 {'entity': 'B-LOC',
  'score': np.float32(0.9975641),
  'index': 6,
  'word': 'paris',
  'start': 28,
  'end': 33},
 {'entity': 'B-PER',
  'score': np.float32(0.99759465),
  'index': 8,
  'word': 'julien',
  'start': 37,
  'end': 43},
 {'entity': 'I-PER',
  'score': np.float32(0.99789536),
  'index': 9,
  'word': 'cha',
  'start': 44,
  'end': 47},
 {'entity': 'I-PER',
  'score': np.float32(0.99737144),
  'index': 10,
  'word': '##um',
  'start': 47,
  'end': 49},
 {'entity': 'I-PER',
  'score': np.float32(0.99581134),
  'index': 11,
  'word': '##ond',
  'start': 49,
  'end': 52}]